<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Custom-функции-для-Cross-validation-и-WAPE" data-toc-modified-id="Custom-функции-для-Cross-validation-и-WAPE-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Custom функции для Cross validation и WAPE</a></span></li><li><span><a href="#Полный-датасет-(без-разделения)---CatBoost" data-toc-modified-id="Полный-датасет-(без-разделения)---CatBoost-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Полный датасет (без разделения) - CatBoost</a></span><ul class="toc-item"><li><span><a href="#Загрузка-и-обработка-данных" data-toc-modified-id="Загрузка-и-обработка-данных-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Загрузка и обработка данных</a></span></li><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Обучение модели</a></span></li><li><span><a href="#Проверка-на-тестовых-данных" data-toc-modified-id="Проверка-на-тестовых-данных-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Проверка на тестовых данных</a></span></li><li><span><a href="#Формирование-итогового-файла-с-прогнозами" data-toc-modified-id="Формирование-итогового-файла-с-прогнозами-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Формирование итогового файла с прогнозами</a></span></li></ul></li></ul></div>

# Построение моделей

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import math

import seaborn as sns
sns.set_palette("pastel")

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.metrics import make_scorer
import catboost as cb
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_score

import json
import warnings

## Custom функции для Cross validation и WAPE

Создадим custom cross validation для Catboost: три итерации выбора по 14 дней для прогноза в качестве валидационных данных.

Объект валидации генерирует индексы для выбора данных из полного датасета (на основании дат). Индексы были получены при сохранении файлов после препроцессинга в книге EDA and Preprocessing

In [2]:
class CustomCrossValidation:
    
    def __init__(self, dict_cv):
        self.dict_cv = dict_cv
    
    def split(self, X=None, y=None, groups=None):
        for i in range(1, 4):
            training_indices = self.dict_cv[f'train_{i}']
            test_indices = self.dict_cv[f'valid_{i}']  

            yield training_indices, test_indices

Создадим метрику

In [3]:
#функция WAPE
def wape(y_true: np.array, y_pred: np.array):
    return np.sum(np.abs(y_true-y_pred))/np.sum(np.abs(y_true))

WAPE в формате catboost

In [4]:
#функция WAPE для catboost
class custom_wape:
    
    def get_final_error(self,error,weight):
        return error
    
    def is_max_optimal(self):
        return False
    
    def evaluate(self, approxes, target, weight):
        
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        
        y_true = target
        y_pred = approxes[0]
        
#         print(y_true)
#         print(y_pred)
        
        # Check for NaN or infinite values
        if np.any(np.isnan(y_true)) or np.any(np.isnan(y_pred)) or np.any(np.isinf(y_true)) or np.any(np.isinf(y_pred)):
            raise ValueError("NaN or infinite values in y_true or y_pred")
        
        return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)), 0.0

___

## Полный датасет (без разделения) - CatBoost

### Загрузка и обработка данных

Загрузим полный датасет для обучения (сохранили в notebook с EDA), а также файл для submission

In [5]:
warnings.filterwarnings('ignore')

#define path to files folder
PATH = 'files/01_01_full_data_catboost/'

#загрузка полного датасета и словаря с индексами для CV
data_train = pd.read_csv(PATH + 'data_train.csv')
data_train_dates = pd.read_csv(PATH + 'data_train_dates.csv')

data_submission = pd.read_csv(PATH + 'data_submission.csv')
data_submission_dates = pd.read_csv(PATH + 'data_submission_dates.csv')

# Opening JSON file
with open(PATH + 'data_train_dict.json') as json_file:
    data_train_cv = json.load(json_file)
json_file.close()

del PATH

Выделим датасет с train (train+valid) данными

In [6]:
#get corresponding indexes
train_valid_idx = data_train_cv['train_1'] + data_train_cv['train_2'] + data_train_cv['train_3']

#select corresponding rows
X_train = data_train[data_train.index.isin(train_valid_idx)]

#distinguish target from features
X_train = data_train.drop(columns=['target'])
y_train = data_train['target']

del train_valid_idx

Выделим фичи категориальные и числовые (категориальные далее понадобятся для Catboost)

In [7]:
%%capture

#create an instance of scaling and encoding the features

#data transformation objects
#using OE (not OHE) for tree-like models (incl boosting) because two much unuique values -> 
#to many transformed columns -> longer time and OHE for linear regression
# sc = StandardScaler()
# oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
# ohe = OneHotEncoder(handle_unknown='ignore')

#list of numeric and non-numeric features
    #days_ny, days_paskha - not encoding      

features_n = ['day',
            'dayofweek',
            'ma_3d',
            'ma_6d',
            'ma_9d',
            'ma_12d',
            'ma_15d',
            'max_3d',
            'max_6d',
            'max_9d',
            'max_12d',
            'max_15d',
            'min_3d',
            'min_6d',
            'min_9d',
            'min_12d',
            'min_15d',
            'sales_0days_ago',
            'sales_1days_ago',
             'sales_2days_ago',
             'sales_3days_ago',
             'sales_4days_ago',
             'sales_5days_ago',
             'sales_6days_ago',
             'sales_7days_ago',
             'sales_8days_ago',
             'sales_9days_ago',
             'sales_10days_ago',
             'sales_11days_ago',
             'sales_12days_ago',
             'sales_13days_ago',
             'sales_14days_ago',
             'sales_15days_ago',
             'sales_16days_ago',
             'sales_17days_ago',
             'sales_18days_ago',
             'sales_19days_ago',
             'sales_20days_ago',
             'sales_21days_ago']

features_str = ['st_id',
                'pr_sku_id',
                'st_city_id',
                'st_division_code',
                'st_type_format_id',
                'st_type_loc_id',
                'st_type_size_id',
                'pr_group_id',
                'pr_cat_id',
                'pr_subcat_id',
                'pr_uom_id',
                'holiday',
               ]

#change all categorical to string
for col in X_train[features_str].columns:
    X_train[col] = X_train[col].astype(str)

###  Обучение модели

Обучим модель, перебирая различные гиперпараметры:

In [10]:
%%time

#define CatBoost model
model = cb.CatBoostRegressor(random_state=99,
                             eval_metric=custom_wape(),
                             iterations=1000,
                             early_stopping_rounds=30)

#create catboost Pool object
ds = cb.Pool(X_train, label=y_train, cat_features=features_str)

#dict with hyperparameters
hp = {'depth': [5, 10, 20, 50, 100],
            'l2_leaf_reg': [1, 3, 5]}

#create instance of cross validation
custom_cv = CustomCrossValidation(dict_cv=data_train_cv).split()

#lauch grid search
grid_search_result  = model.grid_search(param_grid=hp, X=ds, cv=custom_cv, shuffle=False, verbose=True)

0:	learn: 0.9637459	test: 0.9693704	best: 0.9693704 (0)	total: 5.07s	remaining: 1h 24m 27s
1:	learn: 0.9480422	test: 0.9604318	best: 0.9604318 (1)	total: 7.73s	remaining: 1h 4m 16s
2:	learn: 0.9230714	test: 0.9408504	best: 0.9408504 (2)	total: 10.4s	remaining: 57m 32s
3:	learn: 0.9104356	test: 0.9337173	best: 0.9337173 (3)	total: 12.9s	remaining: 53m 44s
4:	learn: 0.8841071	test: 0.9101954	best: 0.9101954 (4)	total: 15.1s	remaining: 50m 9s
5:	learn: 0.8693403	test: 0.8986603	best: 0.8986603 (5)	total: 17.9s	remaining: 49m 18s
6:	learn: 0.8632492	test: 0.8980077	best: 0.8980077 (6)	total: 20.7s	remaining: 48m 52s
7:	learn: 0.8485922	test: 0.8862252	best: 0.8862252 (7)	total: 24.5s	remaining: 50m 37s
8:	learn: 0.8319080	test: 0.8718379	best: 0.8718379 (8)	total: 26.7s	remaining: 49m
9:	learn: 0.8235051	test: 0.8661142	best: 0.8661142 (9)	total: 29.1s	remaining: 48m 5s
10:	learn: 0.8302956	test: 0.8767304	best: 0.8661142 (9)	total: 31.9s	remaining: 47m 45s
11:	learn: 0.8297469	test: 0.880

92:	learn: 0.7171653	test: 1.0633976	best: 0.7845100 (77)	total: 4m 14s	remaining: 41m 21s
93:	learn: 0.7173495	test: 1.0636918	best: 0.7845100 (77)	total: 4m 17s	remaining: 41m 22s
94:	learn: 0.7173098	test: 1.0637192	best: 0.7845100 (77)	total: 4m 20s	remaining: 41m 22s
95:	learn: 0.7172594	test: 1.0637283	best: 0.7845100 (77)	total: 4m 23s	remaining: 41m 20s
96:	learn: 0.7154168	test: 1.0617728	best: 0.7845100 (77)	total: 4m 26s	remaining: 41m 16s
97:	learn: 0.7142054	test: 1.1172938	best: 0.7845100 (77)	total: 4m 28s	remaining: 41m 10s
98:	learn: 0.7144179	test: 1.1175892	best: 0.7845100 (77)	total: 4m 31s	remaining: 41m 9s
99:	learn: 0.7137810	test: 1.1167888	best: 0.7845100 (77)	total: 4m 34s	remaining: 41m 6s
100:	learn: 0.7136746	test: 1.1169201	best: 0.7845100 (77)	total: 4m 36s	remaining: 41m 2s
101:	learn: 0.7135358	test: 1.1168033	best: 0.7845100 (77)	total: 4m 40s	remaining: 41m 7s
102:	learn: 0.7134936	test: 1.1167810	best: 0.7845100 (77)	total: 4m 42s	remaining: 41m 3s
1

73:	learn: 0.8581834	test: 0.8903773	best: 0.8903773 (73)	total: 3m 35s	remaining: 44m 55s
74:	learn: 0.8568294	test: 0.8893012	best: 0.8893012 (74)	total: 3m 39s	remaining: 45m 2s
75:	learn: 0.8571513	test: 0.8903326	best: 0.8893012 (74)	total: 3m 42s	remaining: 45m 5s
76:	learn: 0.8552400	test: 0.8885563	best: 0.8885563 (76)	total: 3m 46s	remaining: 45m 9s
77:	learn: 0.8536089	test: 0.8871650	best: 0.8871650 (77)	total: 3m 48s	remaining: 45m 6s
78:	learn: 0.8519830	test: 0.8856956	best: 0.8856956 (78)	total: 3m 52s	remaining: 45m 6s
79:	learn: 0.8516231	test: 0.8858509	best: 0.8856956 (78)	total: 3m 55s	remaining: 45m 6s
80:	learn: 0.8515168	test: 0.8862637	best: 0.8856956 (78)	total: 3m 58s	remaining: 45m 5s
81:	learn: 0.8517172	test: 0.8871536	best: 0.8856956 (78)	total: 4m 1s	remaining: 45m 4s
82:	learn: 0.8499712	test: 0.8855922	best: 0.8855922 (82)	total: 4m 6s	remaining: 45m 23s
83:	learn: 0.8484093	test: 0.8843243	best: 0.8843243 (83)	total: 4m 11s	remaining: 45m 38s
84:	learn

25:	learn: 0.9925541	test: 0.9942529	best: 0.9942529 (25)	total: 1m 11s	remaining: 44m 24s
26:	learn: 0.9921952	test: 0.9939498	best: 0.9939498 (26)	total: 1m 14s	remaining: 44m 34s
27:	learn: 0.9920145	test: 0.9938401	best: 0.9938401 (27)	total: 1m 17s	remaining: 44m 46s
28:	learn: 0.9916568	test: 0.9935351	best: 0.9935351 (28)	total: 1m 20s	remaining: 45m 4s
29:	learn: 0.9912655	test: 0.9931921	best: 0.9931921 (29)	total: 1m 23s	remaining: 45m 3s
30:	learn: 0.9911125	test: 0.9931102	best: 0.9931102 (30)	total: 1m 26s	remaining: 45m 3s
31:	learn: 0.9907521	test: 0.9927985	best: 0.9927985 (31)	total: 1m 29s	remaining: 45m 5s
32:	learn: 0.9904190	test: 0.9925227	best: 0.9925227 (32)	total: 1m 32s	remaining: 45m 5s
33:	learn: 0.9902569	test: 0.9924494	best: 0.9924494 (33)	total: 1m 35s	remaining: 45m
34:	learn: 0.9900930	test: 0.9923625	best: 0.9923625 (34)	total: 1m 37s	remaining: 44m 58s
35:	learn: 0.9897603	test: 0.9920868	best: 0.9920868 (35)	total: 1m 41s	remaining: 45m 24s
36:	lear

116:	learn: 0.9696677	test: 0.9766625	best: 0.9766625 (116)	total: 5m 9s	remaining: 38m 53s
117:	learn: 0.9695782	test: 0.9766629	best: 0.9766625 (116)	total: 5m 11s	remaining: 38m 50s
118:	learn: 0.9694114	test: 0.9765611	best: 0.9765611 (118)	total: 5m 14s	remaining: 38m 47s
119:	learn: 0.9691116	test: 0.9763032	best: 0.9763032 (119)	total: 5m 17s	remaining: 38m 46s
120:	learn: 0.9687949	test: 0.9760390	best: 0.9760390 (120)	total: 5m 20s	remaining: 38m 45s
121:	learn: 0.9686300	test: 0.9759361	best: 0.9759361 (121)	total: 5m 22s	remaining: 38m 42s
122:	learn: 0.9683546	test: 0.9757236	best: 0.9757236 (122)	total: 5m 25s	remaining: 38m 39s
123:	learn: 0.9682096	test: 0.9756397	best: 0.9756397 (123)	total: 5m 27s	remaining: 38m 36s
124:	learn: 0.9678604	test: 0.9753308	best: 0.9753308 (124)	total: 5m 30s	remaining: 38m 34s
125:	learn: 0.9676324	test: 0.9751472	best: 0.9751472 (125)	total: 5m 33s	remaining: 38m 33s
126:	learn: 0.9674768	test: 0.9750501	best: 0.9750501 (126)	total: 5m 3

...

367:	learn: 0.9896825	test: 0.9921710	best: 0.9921710 (367)	total: 15m 37s	remaining: 26m 49s
368:	learn: 0.9896718	test: 0.9921707	best: 0.9921707 (368)	total: 15m 39s	remaining: 26m 46s
369:	learn: 0.9896402	test: 0.9921442	best: 0.9921442 (369)	total: 15m 42s	remaining: 26m 44s
370:	learn: 0.9896057	test: 0.9921148	best: 0.9921148 (370)	total: 15m 44s	remaining: 26m 41s
371:	learn: 0.9895697	test: 0.9920836	best: 0.9920836 (371)	total: 15m 46s	remaining: 26m 38s
372:	learn: 0.9895347	test: 0.9920541	best: 0.9920541 (372)	total: 15m 49s	remaining: 26m 36s
373:	learn: 0.9895003	test: 0.9920254	best: 0.9920254 (373)	total: 15m 52s	remaining: 26m 33s
374:	learn: 0.9894673	test: 0.9920186	best: 0.9920186 (374)	total: 15m 54s	remaining: 26m 31s
375:	learn: 0.9894315	test: 0.9919873	best: 0.9919873 (375)	total: 15m 57s	remaining: 26m 28s
376:	learn: 0.9894084	test: 0.9919697	best: 0.9919697 (376)	total: 15m 59s	remaining: 26m 25s
377:	learn: 0.9893733	test: 0.9919400	best: 0.9919400 (377)	

455:	learn: 0.9871208	test: 0.9902070	best: 0.9902070 (455)	total: 19m 18s	remaining: 23m 2s
456:	learn: 0.9870802	test: 0.9901693	best: 0.9901693 (456)	total: 19m 20s	remaining: 22m 59s
457:	learn: 0.9870421	test: 0.9901356	best: 0.9901356 (457)	total: 19m 23s	remaining: 22m 56s
458:	learn: 0.9870089	test: 0.9901072	best: 0.9901072 (458)	total: 19m 25s	remaining: 22m 53s
459:	learn: 0.9869842	test: 0.9900876	best: 0.9900876 (459)	total: 19m 27s	remaining: 22m 50s
460:	learn: 0.9869495	test: 0.9900579	best: 0.9900579 (460)	total: 19m 30s	remaining: 22m 47s
461:	learn: 0.9869363	test: 0.9900535	best: 0.9900535 (461)	total: 19m 32s	remaining: 22m 45s
462:	learn: 0.9869177	test: 0.9900415	best: 0.9900415 (462)	total: 19m 35s	remaining: 22m 43s
463:	learn: 0.9868786	test: 0.9900067	best: 0.9900067 (463)	total: 19m 37s	remaining: 22m 40s
464:	learn: 0.9868436	test: 0.9899772	best: 0.9899772 (464)	total: 19m 40s	remaining: 22m 37s
465:	learn: 0.9868339	test: 0.9899800	best: 0.9899772 (464)	t

543:	learn: 0.9848934	test: 0.9885469	best: 0.9885469 (543)	total: 22m 59s	remaining: 19m 16s
544:	learn: 0.9848795	test: 0.9885394	best: 0.9885394 (544)	total: 23m 2s	remaining: 19m 13s
545:	learn: 0.9848651	test: 0.9885329	best: 0.9885329 (545)	total: 23m 4s	remaining: 19m 11s
546:	learn: 0.9848522	test: 0.9885286	best: 0.9885286 (546)	total: 23m 7s	remaining: 19m 8s
547:	learn: 0.9848124	test: 0.9884929	best: 0.9884929 (547)	total: 23m 9s	remaining: 19m 6s
548:	learn: 0.9847823	test: 0.9884690	best: 0.9884690 (548)	total: 23m 12s	remaining: 19m 3s
549:	learn: 0.9847448	test: 0.9884361	best: 0.9884361 (549)	total: 23m 14s	remaining: 19m 1s
550:	learn: 0.9847082	test: 0.9884039	best: 0.9884039 (550)	total: 23m 17s	remaining: 18m 58s
551:	learn: 0.9846674	test: 0.9883668	best: 0.9883668 (551)	total: 23m 19s	remaining: 18m 55s
552:	learn: 0.9846261	test: 0.9883296	best: 0.9883296 (552)	total: 23m 22s	remaining: 18m 53s
553:	learn: 0.9845937	test: 0.9883032	best: 0.9883032 (553)	total: 2

631:	learn: 0.9824689	test: 0.9866399	best: 0.9866399 (631)	total: 26m 42s	remaining: 15m 33s
632:	learn: 0.9824554	test: 0.9866347	best: 0.9866347 (632)	total: 26m 45s	remaining: 15m 30s
633:	learn: 0.9824449	test: 0.9866343	best: 0.9866343 (633)	total: 26m 48s	remaining: 15m 28s
634:	learn: 0.9824099	test: 0.9866045	best: 0.9866045 (634)	total: 26m 50s	remaining: 15m 25s
635:	learn: 0.9823714	test: 0.9865693	best: 0.9865693 (635)	total: 26m 53s	remaining: 15m 23s
636:	learn: 0.9823575	test: 0.9865638	best: 0.9865638 (636)	total: 26m 56s	remaining: 15m 21s
637:	learn: 0.9823470	test: 0.9865633	best: 0.9865633 (637)	total: 26m 59s	remaining: 15m 18s
638:	learn: 0.9823158	test: 0.9865379	best: 0.9865379 (638)	total: 27m 2s	remaining: 15m 16s
639:	learn: 0.9822782	test: 0.9865039	best: 0.9865039 (639)	total: 27m 5s	remaining: 15m 14s
640:	learn: 0.9822493	test: 0.9864805	best: 0.9864805 (640)	total: 27m 8s	remaining: 15m 12s
641:	learn: 0.9822167	test: 0.9864530	best: 0.9864530 (641)	tot

KeyboardInterrupt: 

In [ ]:
# cv_results = pd.DataFrame(grid_search_result["cv_results"])

# cv_results.tail()

Тут просто обучение быстрое для понимания

###  Проверка на тестовых данных

In [ ]:
# model.get_feature_importance(train_data)

Проверим поведение лучший модели на тестовых данных

In [ ]:
#тут надо будет переобучить на полном трейне для теста

In [ ]:
warnings.filterwarnings('ignore')

#get test data (drop the target for correct predict with CatBoost)
X_test = data_train[data_train.index.isin(data_train_cv['test'])]
y_test = X_test['target']
X_test = X_test.drop(columns=['target'])

#change all categorical to string
for col in X_test[features_str].columns:
    X_test[col] = X_test[col].astype(str)

#predict values
pred = model.predict(X_test)

#get test dates
test_dates = data_train_dates[data_train_dates.index.isin(data_train_cv['test'])]
test_dates['date']= pd.to_datetime(test_dates['date'], format='%Y-%m-%d')

#add together
X_test['date'] = test_dates
X_test['pred'] = pred
X_test['target'] = y_test

Рассчитаем WAPE и добавим в словарь для сравнения

In [ ]:
x = wape(y_true=y_test, y_pred=pred)

results_wape = []
results_models = []

results_wape.append(x)
results_models = ['full_data_catboost']

График target VS predict

In [ ]:
pivot = pd.pivot_table(data=X_test,
              index='date',
              values=['target', 'pred'],
              aggfunc='sum').sort_values(by='date')

In [ ]:
plt.figure(figsize=(12,5))
sns.lineplot(data=pivot)

График ошибки

In [ ]:
pivot = pd.pivot_table(data=X_test,
              index='date',
              values=['target', 'pred'],
              aggfunc='sum').sort_values(by='date')
pivot['diff'] = pivot['target'] - pivot['pred']
pivot = pivot.drop(columns=['target', 'pred'])

plt.figure(figsize=(12,5))
sns.lineplot(data=pivot)

Проверим поведение лучший модели на тестовых данных - выберем самый крупный магазин и самую крупную группу товаров

In [ ]:
_ = X_test[(X_test['st_id'] == 'c81e728d9d4c2f636f067f89cc14862c') & 
       (X_test['pr_group_id'] == 'c74d97b01eae257e44aa9d5bade97baf')]

pivot = pd.pivot_table(data=_,
              index='date',
              values=['target', 'pred'],
              aggfunc='sum').sort_values(by='date')

plt.figure(figsize=(12,5))
sns.lineplot(data=pivot)

### Формирование итогового файла с прогнозами

In [ ]:
#select data for submission features and target
X_submission = data_submission.copy()
y_submission = X_submission['target']
X_submission = X_submission.drop(columns=['target'])

#change all categorical to string
for col in X_submission[features_str].columns:
    X_submission[col] = X_submission[col].astype(str)

#RETRAIN THE BEST MODEL

#define CatBoost model
model = cb.CatBoostRegressor(random_state=99,
                             eval_metric=custom_wape(),
                             iterations=3)

#create catboost Pool object
ds = cb.Pool(X_train, label=y_train, cat_features=features_str)

#fit model
model.fit(ds)

#make predictions    
pred_submission = model.predict(X_submission)

#reconstruct the submission file
X_submission['target'] = pred_submission
X_submission['date'] = data_submission_dates
sales_submission_ready = X_submission[['st_id', 'pr_sku_id', 'date', 'target']]

#save to csv
sales_submission_ready.to_csv('sales_submission_ready.csv', index=False)

---